In [1]:
# Standard imports for TensorFlow and numpy
import tensorflow as tf
import numpy as np

2025-06-30 14:38:05.735796: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-30 14:38:05.752218: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751308685.769346 2204596 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751308685.774577 2204596 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751308685.787585 2204596 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
# Set project root directory and add `src` to path
import sys
from pathlib import Path

PROJECT_ROOT = '/scratch/edk202/word2gm-fast'
project_root = Path(PROJECT_ROOT)
src_path = project_root / 'src'

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Optionally, print sys.path for debugging
print('sys.path:', sys.path[:3], '...')

sys.path: ['/scratch/edk202/word2gm-fast/src', '/ext3/miniforge3/bin', '/ext3/miniforge3/condabin'] ...


In [3]:
# Data loading and pipeline setup
from word2gm_fast.utils.tfrecord_io import load_pipeline_artifacts
import os
from pathlib import Path

# Define paths for your small corpus artifacts and output
dataset_artifacts_dir = '/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data/1750_artifacts'
output_dir = '/scratch/edk202/word2gm-fast/output/test_small_corpus'
Path(output_dir).mkdir(parents=True, exist_ok=True)

# Load pipeline artifacts (vocab, triplets, etc.)
artifacts = load_pipeline_artifacts(dataset_artifacts_dir)
vocab_table = artifacts['vocab_table']
triplets_ds = artifacts['triplets_ds']
vocab_size = artifacts['vocab_size']

# Build the dataset pipeline: cache -> shuffle -> batch -> prefetch
triplets_ds = triplets_ds.cache()
BATCH_SIZE = 256
SHUFFLE_BUFFER_SIZE = BATCH_SIZE * 10
triplets_ds = triplets_ds.shuffle(SHUFFLE_BUFFER_SIZE)
triplets_ds = triplets_ds.batch(BATCH_SIZE)
triplets_ds = triplets_ds.prefetch(tf.data.AUTOTUNE)

print(f'Loaded vocab_size: {vocab_size}')

<pre>Loading pipeline artifacts from: /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data/1750_artifacts</pre>

<pre>Loading vocabulary TFRecord from: /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data/1750_artifacts/vocab.tfrecord</pre>

2025-06-30 14:38:08.655689: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-06-30 14:38:08.754488: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:387] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 134217728
2025-06-30 14:38:08.797121: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


<pre>Loading triplet TFRecord from: /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data/1750_artifacts/triplets.tfrecord</pre>

<pre>Triplet TFRecord loaded and parsed</pre>

<pre>All artifacts loaded successfully!</pre>

Loaded vocab_size: 5143


In [4]:
# Launch Word2GM training using notebook_training.py with standard hyperparameters
from word2gm_fast.training.notebook_training import run_notebook_training
from word2gm_fast.models.config import Word2GMConfig

# Construct config if not already defined
try:
    config
except NameError:
    config = Word2GMConfig(
        vocab_size=vocab_size,
        embedding_size=30,
        num_mixtures=2
    )

run_notebook_training(
    training_dataset=triplets_ds,
    save_path=output_dir,
    vocab_size=config.vocab_size,
    embedding_size=config.embedding_size,
    num_mixtures=config.num_mixtures,
    spherical=config.spherical,
    learning_rate=config.learning_rate,
    epochs=3,  # or config.epochs_to_train
    adagrad=config.adagrad,
    normclip=config.normclip,
    norm_cap=config.norm_cap,
    lower_sig=config.lower_sig,
    upper_sig=config.upper_sig,
    wout=config.wout,
    tensorboard_log_path=None,
    monitor_interval=10,
    profile=False,
    var_scale=config.var_scale,
    loss_epsilon=config.loss_epsilon
)